In [ ]:
import keras
from keras import layers
import keras_tuner as kt
import numpy as np

# Define a function that builds the model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))  # Adjust input shape based on your dataset

    # Tune the number of layers
    for i in range(hp.Int("num_layers", 1, 5)):
        # Tune the number of units separately for each layer
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice(f"activation_{i}", ["relu", "tanh", "sigmoid", "leaky_relu"]),
            )
        )

        # Tune batch normalization after each layer
        if hp.Boolean(f"batch_norm_{i}"):
            model.add(layers.BatchNormalization())

        # Tune whether to use dropout after each layer
        if hp.Boolean(f"dropout_{i}"):
            model.add(layers.Dropout(rate=hp.Float(f"dropout_rate_{i}", 0.0, 0.5, step=0.1)))

    # Output layer (adjust number of units to your number of classes)
    model.add(layers.Dense(10, activation="softmax"))  # Modify this line if you have more than 10 classes

    # Tune the optimizer type and its learning rate
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad', 'nadam'])

    if optimizer == 'adam':
        chosen_optimizer = keras.optimizers.Adam(
            learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        )
    elif optimizer == 'rmsprop':
        chosen_optimizer = keras.optimizers.RMSprop(
            learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        )
    elif optimizer == 'sgd':
        chosen_optimizer = keras.optimizers.SGD(
            learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"),
            momentum=hp.Float("momentum", min_value=0.0, max_value=0.9, step=0.1)
        )
    elif optimizer == 'adagrad':
        chosen_optimizer = keras.optimizers.Adagrad(
            learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        )
    elif optimizer == 'nadam':
        chosen_optimizer = keras.optimizers.Nadam(
            learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        )

    model.compile(
        optimizer=chosen_optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=20,  # You can increase the number of trials to explore more configurations
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="tune_everything",
)

# Print the search space summary
tuner.search_space_summary()

# Prepare your dataset (Replace this with your dataset)
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0

# Ensure y_train and y_val are one-hot encoded
num_classes = 10  # Adjust this to match the number of classes in your dataset
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

# Start the search for the best hyperparameter configuration
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

# Optionally, you can retrain the best model on the entire dataset
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
best_model.fit(x_all, y_all, epochs=10)